### Concept Validation Experiment

In [1]:
import torch

# 检查CUDA是否可用
if torch.cuda.is_available():
    # 获取可用的GPU设备数量
    gpu_count = torch.cuda.device_count()
    print(f"发现 {gpu_count} 个可用的GPU 设备.")

    # 遍历并打印每个GPU设备的名称
    for i in range(gpu_count):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i + 1}: {gpu_name}")
else:
    print("未发现可用的GPU设备.")


发现 1 个可用的GPU 设备.
GPU 1: NVIDIA A100-SXM4-80GB


In [2]:
import copy
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge
from bert_score import score
import statistics
from ast import literal_eval
import functools
import json
import os
import random
import wget
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

random.seed(8888)
torch.manual_seed(8888)
random.seed(8888)
np.random.seed(8888)

if torch.cuda.is_available():
    torch.cuda.manual_seed(8888)
    torch.cuda.manual_seed_all(8888)


from tqdm import tqdm
#from transformers_source.src.transformers.models.llama import LlamaForCausalLM, LlamaTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer

torch.set_grad_enabled(False)
tqdm.pandas()

import sys
sys.path.append("/root/Unlearn_Harry_Potter/dissecting_factual_predictions")

# Utilities
from utils import (
    ModelAndTokenizer,
    make_inputs,
    decode_tokens,
    find_token_range,
    predict_from_input,
)

In [3]:
torch.cuda.set_device(0)
# Get CounterFact data for GPT2-xl, from the ROME repository.
#wget.download("https://rome.baulab.info/data/dsets/known_1000.json")



model_name = "/root/autodl-tmp/transformers/OLMo-7B" 
mt = ModelAndTokenizer(
    model_name,
    low_cpu_mem_usage=False,
    torch_dtype=None,
)
mt.model.eval()


The repository for /root/autodl-tmp/transformers/OLMo-7B contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//root/autodl-tmp/transformers/OLMo-7B.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


OLMoForCausalLM(
  (model): Olmo(
    (transformer): ModuleDict(
      (wte): Embedding(50304, 4096)
      (emb_drop): Dropout(p=0.0, inplace=False)
      (ln_f): LayerNorm()
      (blocks): ModuleList(
        (0-31): 32 x OlmoSequentialBlock(
          (dropout): Dropout(p=0.0, inplace=False)
          (act): SwiGLU()
          (attn_out): Linear(in_features=4096, out_features=4096, bias=False)
          (ff_out): Linear(in_features=11008, out_features=4096, bias=False)
          (rotary_emb): RotaryEmbedding()
          (attn_norm): LayerNorm()
          (ff_norm): LayerNorm()
          (att_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (ff_proj): Linear(in_features=4096, out_features=22016, bias=False)
        )
      )
      (ff_out): Linear(in_features=4096, out_features=50304, bias=False)
    )
  )
)

In [11]:
# Bleu Score Calculation
def calculate_bleu(reference, candidate):
    reference = [reference.split()]
    candidate = candidate.split()
    cc = SmoothingFunction()
    bleu_score = sentence_bleu(reference, candidate, smoothing_function=cc.method3)
    return bleu_score

# Rouge-L Score Calculation
def calculate_rouge_l(reference, candidate):
    # print('reference: ',reference)
    # print('candidate: ',candidate)
    rouge = Rouge()
    scores = rouge.get_scores(candidate, reference)
    rouge_l_score = scores[0]['rouge-l']['f']
    return rouge_l_score

# BLEURT-20 Score Calculation
def calculate_bleurt(references, candidates, model, tokenizer, config):

    model.eval()
    with torch.no_grad():
        inputs = mt.tokenizer(references, candidates, padding='longest', return_tensors='pt')
        res = model(**inputs).logits.flatten().tolist()
    print(res)
    return res

    
# Bert-F Score Calculation
def calculate_bert_f(reference, candidate):
    _, _, bert_scores = score([candidate], [reference], lang="en", model_type="bert-base-uncased")
    bert_f_score = bert_scores[0]  # Extracting the F1 score
    return bert_f_score

def add_noise(location, noise_scale = 0):
    # Create Gaussian noise
    mean = 0
    std = noise_scale
    shape = (4096,)
    
    noise = torch.normal(mean, std, size=shape).to('cuda')
    dimension, layer = location[0], location[1]
    global new_params
    new_params[f'model.transformer.blocks.{layer}.ff_out.weight'].T[dimension,:] += noise 

def answers_generate(Questions, Questions_unrelated, location, noise = 0):
    answers_list = []
    unrelated_answers_list = []

    if noise != 0:
        add_noise(location = location, noise_scale = noise)
        global old_params, new_params
        mt.model.load_state_dict(new_params)

    len_questions = len(Questions)
    for idx, question in enumerate(Questions + Questions_unrelated):
        inputs = mt.tokenizer(f"Question: {question}\n Answer:", return_tensors="pt")
        input_ids = inputs["input_ids"].to('cuda')
        
        with torch.no_grad():
            generation_output = mt.model.generate(  #mt.model
                input_ids=input_ids,
                return_dict_in_generate=True,
                do_sample = False,
                max_new_tokens=100,
            )
        s = generation_output.sequences[0]
        output = mt.tokenizer.decode(s)
        # Find the index of "Answer:"
        answer_index = output.find("Answer:")
        if answer_index != -1:
            # Extract the text after "Answer:"
            answer_text = output[answer_index + len("Answer:"):].strip()
            if idx < len_questions:
                answers_list.append(answer_text)
            else:  
                unrelated_answers_list.append(answer_text)
            
        else:
            print("Answer not found.")
            
    if noise != 0:
        new_params = old_params
        mt.model.load_state_dict(old_params)

    return answers_list, unrelated_answers_list
    


In [5]:
# Copy model params
global old_params, new_params

old_params = copy.deepcopy(mt.model.state_dict())
new_params = copy.deepcopy(mt.model.state_dict())
#new_params

In [6]:
def random_select_except(lst, n, exclude_index):
    # 排除指定位置的元素
    candidates = [elem for i, elem in enumerate(lst) if i != exclude_index]
    # 从候选元素中随机选择 n 个
    selected = random.sample(candidates, n)
    return selected

In [7]:
from openpyxl import Workbook
import re

# 创建一个工作簿
wb = Workbook()
ws = wb.active
# 设置列名
ws.append(["id", "Concept", "Location", "bleu_score", "unrelated_QA_bleu_score", "rouge_l_score","unrelated_QA_rouge_l_score","original_answers","perturbed_answers", "original_unrelated_answers","perturbed_unrelated_answers"])

In [8]:
with open("/root/Unlearn_Harry_Potter/Baselines/ConceptMap/ConceptMap_data/olmo-7b_concepts/olmo-7b_concepts.json", "r", encoding="utf-8") as file:
    concepts_list = json.load(file)

#concepts = ["Amazon Alexa", "Harry Potter", "The Lord of the Rings", "Super Mario", "Star Wars", "William Shakespeare", "Sherlock Holmes", "Netflix", "Blockchain", "Satan"]
#concepts = ["William Shakespeare", "Netflix",  "Satan"]
#concepts_for_random_sample = ["William Shakespeare", "Netflix",  "Satan"]

for idx,concept in enumerate(concepts_list):
    bleu_scores = []
    rouge_l_scores = []
    bert_f_scores = []
    unrelated_bleu_scores = []
    unrelated_rouge_l_scores = []
    unrelated_bert_f_scores = []
    print("idx: ",idx)
    
    dimension, layer = concept['Dimension'], concept['Layer']

    concept_name = concept['Concept']
    Questions = concept['QA']

    Questions_unrelated = []

    random_selection = random_select_except(concepts_list, 5, idx)
    
    for selection in random_selection:
        Questions_unrelated += selection['QA']
     
    original_answers, original_unrelated_answers = answers_generate(Questions, Questions_unrelated, noise = 0, location = [dimension, layer])
    perturbed_answers, perturbed_unrelated_answers = answers_generate(Questions, Questions_unrelated, noise = 0.1, location = [dimension, layer])
    #perturbed_answers_big = answers_generate(Questions, noise = 0.3)

    for perturbed_answer, original_answer in zip(perturbed_answers, original_answers):
        bleu_scores.append(calculate_bleu(perturbed_answer, original_answer))
        rouge_l_scores.append(calculate_rouge_l(perturbed_answer, original_answer))
        #bert_f_scores.append(calculate_bert_f(perturbed_answer, original_answer))

   
    for perturbed_unrelated_answer, original_unrelated_answer in zip(perturbed_unrelated_answers, original_unrelated_answers):
        unrelated_bleu_scores.append(calculate_bleu(perturbed_unrelated_answer, original_unrelated_answer))
        unrelated_rouge_l_scores.append(calculate_rouge_l(perturbed_unrelated_answer, original_unrelated_answer))
        #unrelated_bert_f_scores.append(calculate_bert_f(perturbed_unrelated_answer, original_unrelated_answer))
    

    bleu_score = statistics.mean(bleu_scores)  
    rouge_l_score = statistics.mean(rouge_l_scores)  
    
    # bert_f_scores = [tensor.item() for tensor in bert_f_scores]
    # bert_f_score = statistics.mean(bert_f_scores)  

    unrelated_bleu_score = statistics.mean(unrelated_bleu_scores)  
    unrelated_rouge_l_score = statistics.mean(unrelated_rouge_l_scores) 

    # unrelated_bert_f_scores = [tensor.item() for tensor in unrelated_bert_f_scores]
    # unrelated_bert_f_score = statistics.mean(unrelated_bert_f_scores) 
    
    print(f"Concept: {concept_name} Validation: ", f"bleu_score: {bleu_score} ", f"rouge_l_score: {rouge_l_score} ")
    print(f"Concept: {concept_name} Validation: ", f"unrelated_bleu_score: {unrelated_bleu_score} ", f"unrelated_rouge_l_score: {unrelated_rouge_l_score} ")
    #print(f"Concept: {concept_name} Validation: ", f"bleu_score: {bleu_score} ", f"rouge_l_score: {rouge_l_score} ")
    #print(f"Concept: {concept_name} Validation: ", f"unrelated_bleu_score: {unrelated_bleu_score} ", f"unrelated_rouge_l_score: {unrelated_rouge_l_score} ")
    row_data = [idx, concept_name, bleu_score, unrelated_bleu_score, rouge_l_score, unrelated_rouge_l_score, str(original_answers),str(perturbed_answers), str(original_unrelated_answers),str(perturbed_unrelated_answers)]
    ws.append(row_data)    
 
wb.save("/root/Unlearn_Harry_Potter/Baselines/ConceptMap/ConceptMap_data/olmo-7b_concepts/olmo-7b_validation_full.xlsx")    

idx:  0
Concept: Amazon (company) Validation:  bleu_score: 0.4995291222536099  rouge_l_score: 0.5953989556704831 
Concept: Amazon (company) Validation:  unrelated_bleu_score: 0.8509865320310801  unrelated_rouge_l_score: 0.9100308793683063 
idx:  1
Concept: Aromatic compound Validation:  bleu_score: 0.12450376335348819  rouge_l_score: 0.342451259953005 
Concept: Aromatic compound Validation:  unrelated_bleu_score: 0.8277908120474662  unrelated_rouge_l_score: 0.899967636234213 
idx:  2
Concept: Internet of things Validation:  bleu_score: 0.046659200490326014  rouge_l_score: 0.24623205127396863 
Concept: Internet of things Validation:  unrelated_bleu_score: 0.8454889641259646  unrelated_rouge_l_score: 0.896533613210974 
idx:  3
Concept: Filmmaking Validation:  bleu_score: 0.7365279902657514  rouge_l_score: 0.8558107109969078 
Concept: Filmmaking Validation:  unrelated_bleu_score: 0.8171916074870171  unrelated_rouge_l_score: 0.8718285835087504 
idx:  4
Concept: Feng shui Validation:  bleu_

In [18]:
with open("/root/Unlearn_Harry_Potter/Baselines/ConceptMap/ConceptMap_data/olmo-7b_concepts/olmo-7b_concepts.json", "r", encoding="utf-8") as file:
    concepts_list = json.load(file)

#concepts = ["Amazon Alexa", "Harry Potter", "The Lord of the Rings", "Super Mario", "Star Wars", "William Shakespeare", "Sherlock Holmes", "Netflix", "Blockchain", "Satan"]
#concepts = ["William Shakespeare", "Netflix",  "Satan"]
#concepts_for_random_sample = ["William Shakespeare", "Netflix",  "Satan"]

for idx,concept in enumerate(concepts_list):
    bleu_scores = []
    rouge_l_scores = []
    bert_f_scores = []
    unrelated_bleu_scores = []
    unrelated_rouge_l_scores = []
    unrelated_bert_f_scores = []

    if concept['ID'] <= 101:
        continue

    if concept['ID'] >= 115:
        break
    
    print("idx: ",idx)
    
    
    dimension, layer = concept['Dimension'], concept['Layer']

    concept_name = concept['Concept']
    Questions = concept['QA']

    Questions_unrelated = []

    random_selection = random_select_except(concepts_list, 5, idx)
    
    for selection in random_selection:
        Questions_unrelated += selection['QA']
     
    original_answers, original_unrelated_answers = answers_generate(Questions, Questions_unrelated, noise = 0, location = [dimension, layer])
    perturbed_answers, perturbed_unrelated_answers = answers_generate(Questions, Questions_unrelated, noise = 0.1, location = [dimension, layer])
    #perturbed_answers_big = answers_generate(Questions, noise = 0.3)

    for perturbed_answer, original_answer in zip(perturbed_answers, original_answers):
        bleu_scores.append(calculate_bleu(perturbed_answer, original_answer))
        rouge_l_scores.append(calculate_rouge_l(perturbed_answer, original_answer))
        #bert_f_scores.append(calculate_bert_f(perturbed_answer, original_answer))

   
    for perturbed_unrelated_answer, original_unrelated_answer in zip(perturbed_unrelated_answers, original_unrelated_answers):
        unrelated_bleu_scores.append(calculate_bleu(perturbed_unrelated_answer, original_unrelated_answer))
        unrelated_rouge_l_scores.append(calculate_rouge_l(perturbed_unrelated_answer, original_unrelated_answer))
        #unrelated_bert_f_scores.append(calculate_bert_f(perturbed_unrelated_answer, original_unrelated_answer))
    

    bleu_score = statistics.mean(bleu_scores)  
    rouge_l_score = statistics.mean(rouge_l_scores)  
    
    # bert_f_scores = [tensor.item() for tensor in bert_f_scores]
    # bert_f_score = statistics.mean(bert_f_scores)  

    unrelated_bleu_score = statistics.mean(unrelated_bleu_scores)  
    unrelated_rouge_l_score = statistics.mean(unrelated_rouge_l_scores) 

    # unrelated_bert_f_scores = [tensor.item() for tensor in unrelated_bert_f_scores]
    # unrelated_bert_f_score = statistics.mean(unrelated_bert_f_scores) 
    
    print(f"Concept: {concept_name} Validation: ", f"bleu_score: {bleu_score} ", f"rouge_l_score: {rouge_l_score} ")
    print(f"Concept: {concept_name} Validation: ", f"unrelated_bleu_score: {unrelated_bleu_score} ", f"unrelated_rouge_l_score: {unrelated_rouge_l_score} ")
    #print(f"Concept: {concept_name} Validation: ", f"bleu_score: {bleu_score} ", f"rouge_l_score: {rouge_l_score} ")
    #print(f"Concept: {concept_name} Validation: ", f"unrelated_bleu_score: {unrelated_bleu_score} ", f"unrelated_rouge_l_score: {unrelated_rouge_l_score} ")
    row_data = [idx, concept_name, bleu_score, unrelated_bleu_score, rouge_l_score, unrelated_rouge_l_score, str(original_answers),str(perturbed_answers), str(original_unrelated_answers),str(perturbed_unrelated_answers)]
    ws.append(row_data)    
 
wb.save("/root/Unlearn_Harry_Potter/Baselines/ConceptMap/ConceptMap_data/olmo-7b_concepts/olmo-7b_validation_full_2.xlsx")    

idx:  102
Concept: Brexit Validation:  bleu_score: 0.5990800330769315  rouge_l_score: 0.7550823654867884 
Concept: Brexit Validation:  unrelated_bleu_score: 0.6643852473108455  unrelated_rouge_l_score: 0.818550871972047 
idx:  103
Concept: Hip hop music Validation:  bleu_score: 0.24401567113437764  rouge_l_score: 0.33644573537675304 
Concept: Hip hop music Validation:  unrelated_bleu_score: 0.6061918057181417  unrelated_rouge_l_score: 0.7724436989553035 
idx:  104
Concept: Amazon Alexa Validation:  bleu_score: 0.18759916037941912  rouge_l_score: 0.44432754959009646 
Concept: Amazon Alexa Validation:  unrelated_bleu_score: 0.7206882179783424  unrelated_rouge_l_score: 0.8348242174881763 
idx:  105
Concept: Barack Obama Validation:  bleu_score: 0.10571620416933594  rouge_l_score: 0.29592733963427365 
Concept: Barack Obama Validation:  unrelated_bleu_score: 0.6144929544501357  unrelated_rouge_l_score: 0.7459796330461458 
idx:  106
Concept: Marathon Validation:  bleu_score: 0.77502342530930

ValueError: Reference is empty.

In [23]:
with open("/root/Unlearn_Harry_Potter/Baselines/ConceptMap/ConceptMap_data/olmo-7b_concepts/olmo-7b_concepts.json", "r", encoding="utf-8") as file:
    concepts_list = json.load(file)

mt.model.load_state_dict(old_params)
#concepts = ["Amazon Alexa", "Harry Potter", "The Lord of the Rings", "Super Mario", "Star Wars", "William Shakespeare", "Sherlock Holmes", "Netflix", "Blockchain", "Satan"]
#concepts = ["William Shakespeare", "Netflix",  "Satan"]
#concepts_for_random_sample = ["William Shakespeare", "Netflix",  "Satan"]

for idx,concept in enumerate(concepts_list):
    bleu_scores = []
    rouge_l_scores = []
    bert_f_scores = []
    unrelated_bleu_scores = []
    unrelated_rouge_l_scores = []
    unrelated_bert_f_scores = []

    if concept['ID'] <= 111:
        continue

    if concept['ID'] >= 115:
        break
    
    print("idx: ",idx)
    
    
    dimension, layer = concept['Dimension'], concept['Layer']

    concept_name = concept['Concept']
    Questions = concept['QA']

    Questions_unrelated = []

    random_selection = random_select_except(concepts_list, 5, idx)
    
    for selection in random_selection:
        Questions_unrelated += selection['QA']
     
    original_answers, original_unrelated_answers = answers_generate(Questions, Questions_unrelated, noise = 0, location = [dimension, layer])
    perturbed_answers, perturbed_unrelated_answers = answers_generate(Questions, Questions_unrelated, noise = 0.1, location = [dimension, layer])
    #perturbed_answers_big = answers_generate(Questions, noise = 0.3)
    print(f"len(original_answers): {len(original_answers)}, len(perturbed_answers): {len(perturbed_answers)}, len(original_unrelated_answers): {len(original_unrelated_answers)}, len(perturbed_unrelated_answers): {len(perturbed_unrelated_answers)}")
    print('original_answers: ',original_answers)
    print('original_unrelated_answers: ',original_unrelated_answers)
    
    for perturbed_answer, original_answer in zip(perturbed_answers, original_answers):
        bleu_scores.append(calculate_bleu(perturbed_answer, original_answer))
        rouge_l_scores.append(calculate_rouge_l(perturbed_answer, original_answer))
        #bert_f_scores.append(calculate_bert_f(perturbed_answer, original_answer))

   
    for perturbed_unrelated_answer, original_unrelated_answer in zip(perturbed_unrelated_answers, original_unrelated_answers):
        unrelated_bleu_scores.append(calculate_bleu(perturbed_unrelated_answer, original_unrelated_answer))
        unrelated_rouge_l_scores.append(calculate_rouge_l(perturbed_unrelated_answer, original_unrelated_answer))
        #unrelated_bert_f_scores.append(calculate_bert_f(perturbed_unrelated_answer, original_unrelated_answer))
    

    bleu_score = statistics.mean(bleu_scores)  
    rouge_l_score = statistics.mean(rouge_l_scores)  
    
    # bert_f_scores = [tensor.item() for tensor in bert_f_scores]
    # bert_f_score = statistics.mean(bert_f_scores)  

    unrelated_bleu_score = statistics.mean(unrelated_bleu_scores)  
    unrelated_rouge_l_score = statistics.mean(unrelated_rouge_l_scores) 

    # unrelated_bert_f_scores = [tensor.item() for tensor in unrelated_bert_f_scores]
    # unrelated_bert_f_score = statistics.mean(unrelated_bert_f_scores) 
    
    print(f"Concept: {concept_name} Validation: ", f"bleu_score: {bleu_score} ", f"rouge_l_score: {rouge_l_score} ")
    print(f"Concept: {concept_name} Validation: ", f"unrelated_bleu_score: {unrelated_bleu_score} ", f"unrelated_rouge_l_score: {unrelated_rouge_l_score} ")
    #print(f"Concept: {concept_name} Validation: ", f"bleu_score: {bleu_score} ", f"rouge_l_score: {rouge_l_score} ")
    #print(f"Concept: {concept_name} Validation: ", f"unrelated_bleu_score: {unrelated_bleu_score} ", f"unrelated_rouge_l_score: {unrelated_rouge_l_score} ")
    row_data = [idx, concept_name, bleu_score, unrelated_bleu_score, rouge_l_score, unrelated_rouge_l_score, str(original_answers),str(perturbed_answers), str(original_unrelated_answers),str(perturbed_unrelated_answers)]
    ws.append(row_data)    
 
wb.save("/root/Unlearn_Harry_Potter/Baselines/ConceptMap/ConceptMap_data/olmo-7b_concepts/olmo-7b_validation_full_2.xlsx")    

idx:  112
len(original_answers): 10, len(perturbed_answers): 10, len(original_unrelated_answers): 50, len(perturbed_unrelated_answers): 50
original_answers:  ['Question:  What is the name of the (A)\n\nAnswer:  (A)\n\nQuestion:  What is the name of the (A)\n\nAnswer:  (A)\n\nQuestion:  What is the name of the (A)\n\nAnswer:  (A)\n\nQuestion:  What is the name of the (A)\n\nAnswer:  (A)\n\nQuestion:  What is the name of the (', 'Question:  What B-A-D is the first-  \nAnswer:  \n\nQuestion:  What is the  \nAnswer:  \n\nQuestion:  What is the  \nAnswer:  \n\nQuestion:  What is the  \nAnswer:  \n\nQuestion:  What is the  \nAnswer:  \n\nQuestion:  What is the  \nAnswer:  \n\nQuestion:  What is the  \nAnswer:', '-  \n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-\n\n-', '-  \n\n-\n\n-\n\nQuestion:  \n\nAnswer:\n\nQuestion:  What\n\nAnswer:\n\nQuestion:  What\n\nAnswer:\n\nQuestion:  What\n\n

In [12]:
with open("/root/Unlearn_Harry_Potter/Baselines/ConceptMap/ConceptMap_data/olmo-7b_concepts/olmo-7b_concepts.json", "r", encoding="utf-8") as file:
    concepts_list = json.load(file)

#concepts = ["Amazon Alexa", "Harry Potter", "The Lord of the Rings", "Super Mario", "Star Wars", "William Shakespeare", "Sherlock Holmes", "Netflix", "Blockchain", "Satan"]
#concepts = ["William Shakespeare", "Netflix",  "Satan"]
#concepts_for_random_sample = ["William Shakespeare", "Netflix",  "Satan"]

for idx,concept in enumerate(concepts_list):
    bleu_scores = []
    rouge_l_scores = []
    bert_f_scores = []
    unrelated_bleu_scores = []
    unrelated_rouge_l_scores = []
    unrelated_bert_f_scores = []

    if concept['ID'] <= 114:
        continue
    
    print("idx: ",idx)
    
    
    dimension, layer = concept['Dimension'], concept['Layer']

    concept_name = concept['Concept']
    Questions = concept['QA']

    Questions_unrelated = []

    random_selection = random_select_except(concepts_list, 5, idx)
    
    for selection in random_selection:
        Questions_unrelated += selection['QA']
     
    original_answers, original_unrelated_answers = answers_generate(Questions, Questions_unrelated, noise = 0, location = [dimension, layer])
    perturbed_answers, perturbed_unrelated_answers = answers_generate(Questions, Questions_unrelated, noise = 0.1, location = [dimension, layer])
    #perturbed_answers_big = answers_generate(Questions, noise = 0.3)

    print('perturbed_answers: ',perturbed_answers)
    print('original_answers: ',original_answers)

    for perturbed_answer, original_answer in zip(perturbed_answers, original_answers):
        bleu_scores.append(calculate_bleu(perturbed_answer, original_answer))
        rouge_l_scores.append(calculate_rouge_l(perturbed_answer, original_answer))
        #bert_f_scores.append(calculate_bert_f(perturbed_answer, original_answer))

   
    for perturbed_unrelated_answer, original_unrelated_answer in zip(perturbed_unrelated_answers, original_unrelated_answers):
        unrelated_bleu_scores.append(calculate_bleu(perturbed_unrelated_answer, original_unrelated_answer))
        unrelated_rouge_l_scores.append(calculate_rouge_l(perturbed_unrelated_answer, original_unrelated_answer))
        #unrelated_bert_f_scores.append(calculate_bert_f(perturbed_unrelated_answer, original_unrelated_answer))
    

    bleu_score = statistics.mean(bleu_scores)  
    rouge_l_score = statistics.mean(rouge_l_scores)  
    
    # bert_f_scores = [tensor.item() for tensor in bert_f_scores]
    # bert_f_score = statistics.mean(bert_f_scores)  

    unrelated_bleu_score = statistics.mean(unrelated_bleu_scores)  
    unrelated_rouge_l_score = statistics.mean(unrelated_rouge_l_scores) 

    # unrelated_bert_f_scores = [tensor.item() for tensor in unrelated_bert_f_scores]
    # unrelated_bert_f_score = statistics.mean(unrelated_bert_f_scores) 
    
    print(f"Concept: {concept_name} Validation: ", f"bleu_score: {bleu_score} ", f"rouge_l_score: {rouge_l_score} ")
    print(f"Concept: {concept_name} Validation: ", f"unrelated_bleu_score: {unrelated_bleu_score} ", f"unrelated_rouge_l_score: {unrelated_rouge_l_score} ")
    #print(f"Concept: {concept_name} Validation: ", f"bleu_score: {bleu_score} ", f"rouge_l_score: {rouge_l_score} ")
    #print(f"Concept: {concept_name} Validation: ", f"unrelated_bleu_score: {unrelated_bleu_score} ", f"unrelated_rouge_l_score: {unrelated_rouge_l_score} ")
    row_data = [idx, concept_name, bleu_score, unrelated_bleu_score, rouge_l_score, unrelated_rouge_l_score, str(original_answers),str(perturbed_answers), str(original_unrelated_answers),str(perturbed_unrelated_answers)]
    ws.append(row_data)    
 
wb.save("/root/Unlearn_Harry_Potter/Baselines/ConceptMap/ConceptMap_data/olmo-7b_concepts/olmo-7b_validation_full_2.xlsx")    

idx:  115
perturbed_answers:  ['Wen, theen, and theen.\n\nQuestion:  What is the name of the first man?\n\n Answer:  Theen.\n\nQuestion:  What is the name of the first woman?\n\n Answer:  Theen.\n\nQuestion:  What is the name of the first child?\n\n Answer:  Theen.\n\nQuestion:  What is the name of the first dog?\n\n Answer:  Theen.\n\nQuestion:', "ists:  Theists:  A  \n\nQuestion:  What is the name of the first man?\n\n Answer:ists:  Theists:  Adam  \n\nQuestion:  What is the name of the first woman?\n\n Answer:ists:  Theists:  Eve  \n\nQuestion:  What is the name of the first man's wife?\n\n Answer:ists:  Theists:    \n\nQuestion:  What is the name of the", 'The\xa0*', 'The\xa0Rap\xa0of\xa0the\xa0Cradle\xa0(or\xa0Rap\xa0of\xa0the\xa0Cradle)\xa0is\xa0a\xa0song\xa0written\xa0by\xa0the\xa0American\xa0poet\xa0and\xa0songwriter\xa0Robert\xa0Bly.\xa0It\xa0was\xa0first\xa0published\xa0in\xa0the\xa0book\xa0The\xa0Rap\xa0of\xa0the\xa0Cradle\xa0in\xa0the\xa0Spring\xa0of\xa0the\xa0Year\xa0(1979

ValueError: Hypothesis is empty.

In [14]:
with open("/root/Unlearn_Harry_Potter/Baselines/ConceptMap/ConceptMap_data/olmo-7b_concepts/olmo-7b_concepts.json", "r", encoding="utf-8") as file:
    concepts_list = json.load(file)

#concepts = ["Amazon Alexa", "Harry Potter", "The Lord of the Rings", "Super Mario", "Star Wars", "William Shakespeare", "Sherlock Holmes", "Netflix", "Blockchain", "Satan"]
#concepts = ["William Shakespeare", "Netflix",  "Satan"]
#concepts_for_random_sample = ["William Shakespeare", "Netflix",  "Satan"]

for idx,concept in enumerate(concepts_list):
    bleu_scores = []
    rouge_l_scores = []
    bert_f_scores = []
    unrelated_bleu_scores = []
    unrelated_rouge_l_scores = []
    unrelated_bert_f_scores = []

    if concept['ID'] <= 137:
        continue
    
    print("idx: ",idx)
    
    
    dimension, layer = concept['Dimension'], concept['Layer']

    concept_name = concept['Concept']
    Questions = concept['QA']

    Questions_unrelated = []

    random_selection = random_select_except(concepts_list, 5, idx)
    
    for selection in random_selection:
        Questions_unrelated += selection['QA']
     
    original_answers, original_unrelated_answers = answers_generate(Questions, Questions_unrelated, noise = 0, location = [dimension, layer])
    perturbed_answers, perturbed_unrelated_answers = answers_generate(Questions, Questions_unrelated, noise = 0.1, location = [dimension, layer])
    #perturbed_answers_big = answers_generate(Questions, noise = 0.3)

    print(f'len(perturbed_answers): {len(perturbed_answers)}: ',f'len(original_answers): {len(original_answers)}')
    print('perturbed_answers: ',perturbed_answers)
    print('original_answers: ',original_answers)

    for perturbed_answer, original_answer in zip(perturbed_answers, original_answers):
        bleu_scores.append(calculate_bleu(perturbed_answer, original_answer))
        rouge_l_scores.append(calculate_rouge_l(perturbed_answer, original_answer))
        #bert_f_scores.append(calculate_bert_f(perturbed_answer, original_answer))

   
    for perturbed_unrelated_answer, original_unrelated_answer in zip(perturbed_unrelated_answers, original_unrelated_answers):
        unrelated_bleu_scores.append(calculate_bleu(perturbed_unrelated_answer, original_unrelated_answer))
        unrelated_rouge_l_scores.append(calculate_rouge_l(perturbed_unrelated_answer, original_unrelated_answer))
        #unrelated_bert_f_scores.append(calculate_bert_f(perturbed_unrelated_answer, original_unrelated_answer))
    

    bleu_score = statistics.mean(bleu_scores)  
    rouge_l_score = statistics.mean(rouge_l_scores)  
    
    # bert_f_scores = [tensor.item() for tensor in bert_f_scores]
    # bert_f_score = statistics.mean(bert_f_scores)  

    unrelated_bleu_score = statistics.mean(unrelated_bleu_scores)  
    unrelated_rouge_l_score = statistics.mean(unrelated_rouge_l_scores) 

    # unrelated_bert_f_scores = [tensor.item() for tensor in unrelated_bert_f_scores]
    # unrelated_bert_f_score = statistics.mean(unrelated_bert_f_scores) 
    
    print(f"Concept: {concept_name} Validation: ", f"bleu_score: {bleu_score} ", f"rouge_l_score: {rouge_l_score} ")
    print(f"Concept: {concept_name} Validation: ", f"unrelated_bleu_score: {unrelated_bleu_score} ", f"unrelated_rouge_l_score: {unrelated_rouge_l_score} ")
    #print(f"Concept: {concept_name} Validation: ", f"bleu_score: {bleu_score} ", f"rouge_l_score: {rouge_l_score} ")
    #print(f"Concept: {concept_name} Validation: ", f"unrelated_bleu_score: {unrelated_bleu_score} ", f"unrelated_rouge_l_score: {unrelated_rouge_l_score} ")
    row_data = [idx, concept_name, bleu_score, unrelated_bleu_score, rouge_l_score, unrelated_rouge_l_score, str(original_answers),str(perturbed_answers), str(original_unrelated_answers),str(perturbed_unrelated_answers)]
    ws.append(row_data)    
 
wb.save("/root/Unlearn_Harry_Potter/Baselines/ConceptMap/ConceptMap_data/olmo-7b_concepts/olmo-7b_validation_full_2.xlsx")    

idx:  138
len(perturbed_answers): 10:  len(original_answers): 10
perturbed_answers:  ['To get the next [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [B [', '[2](https://www.youtube.com/watch?v=_ZjYqZjYqZg)\n\nQuestion:  How many players are on each team in a basketball game?\n\n Answer:  [5](https://www.youtube.com/watch?v=_ZjYqZjYqZg)\n\nQuestion:  How many players are on each team in a football game?\n\n Answer:  [ [1', '[https://www.youtube.com/watch?v=1\\_Y\\_Q\\_Y\\_Q\\_Y](https://www.youtube.com/watch?v=1_Y_Q_Y_Q_Y)<|endoftext|>', '[ soo...\n\nhttps://www.google.com/search?q=o [1]\n\nhttps://www.google.com/search?q= [2]\n\nhttps://www.google.com/search?q= [3]\n\nhttps://www.google.com/search?q= [4]\n\nhttps://www.google.com/search?q= [5]\n\nhttps://www.google.com/', '[https://www.youtube.com/watch?v=1\\_Y\\_Q\\_Y\\_Q\\_Y](https://www.youtube.com/watch?v=1_Y_Q_Y_Q_Y)<|endoftext|>', '[ left 

ValueError: Hypothesis is empty.

In [15]:
with open("/root/Unlearn_Harry_Potter/Baselines/ConceptMap/ConceptMap_data/olmo-7b_concepts/olmo-7b_concepts.json", "r", encoding="utf-8") as file:
    concepts_list = json.load(file)

#concepts = ["Amazon Alexa", "Harry Potter", "The Lord of the Rings", "Super Mario", "Star Wars", "William Shakespeare", "Sherlock Holmes", "Netflix", "Blockchain", "Satan"]
#concepts = ["William Shakespeare", "Netflix",  "Satan"]
#concepts_for_random_sample = ["William Shakespeare", "Netflix",  "Satan"]

for idx,concept in enumerate(concepts_list):
    bleu_scores = []
    rouge_l_scores = []
    bert_f_scores = []
    unrelated_bleu_scores = []
    unrelated_rouge_l_scores = []
    unrelated_bert_f_scores = []

    if concept['ID'] <= 159:
        continue
    
    print("idx: ",idx)
    
    
    dimension, layer = concept['Dimension'], concept['Layer']

    concept_name = concept['Concept']
    Questions = concept['QA']

    Questions_unrelated = []

    random_selection = random_select_except(concepts_list, 5, idx)
    
    for selection in random_selection:
        Questions_unrelated += selection['QA']
     
    original_answers, original_unrelated_answers = answers_generate(Questions, Questions_unrelated, noise = 0, location = [dimension, layer])
    perturbed_answers, perturbed_unrelated_answers = answers_generate(Questions, Questions_unrelated, noise = 0.1, location = [dimension, layer])
    #perturbed_answers_big = answers_generate(Questions, noise = 0.3)

    print(f'len(perturbed_answers): {len(perturbed_answers)}: ',f'len(original_answers): {len(original_answers)}')
    print('perturbed_answers: ',perturbed_answers)
    print('original_answers: ',original_answers)

    for perturbed_answer, original_answer in zip(perturbed_answers, original_answers):
        bleu_scores.append(calculate_bleu(perturbed_answer, original_answer))
        rouge_l_scores.append(calculate_rouge_l(perturbed_answer, original_answer))
        #bert_f_scores.append(calculate_bert_f(perturbed_answer, original_answer))

   
    for perturbed_unrelated_answer, original_unrelated_answer in zip(perturbed_unrelated_answers, original_unrelated_answers):
        unrelated_bleu_scores.append(calculate_bleu(perturbed_unrelated_answer, original_unrelated_answer))
        unrelated_rouge_l_scores.append(calculate_rouge_l(perturbed_unrelated_answer, original_unrelated_answer))
        #unrelated_bert_f_scores.append(calculate_bert_f(perturbed_unrelated_answer, original_unrelated_answer))
    

    bleu_score = statistics.mean(bleu_scores)  
    rouge_l_score = statistics.mean(rouge_l_scores)  
    
    # bert_f_scores = [tensor.item() for tensor in bert_f_scores]
    # bert_f_score = statistics.mean(bert_f_scores)  

    unrelated_bleu_score = statistics.mean(unrelated_bleu_scores)  
    unrelated_rouge_l_score = statistics.mean(unrelated_rouge_l_scores) 

    # unrelated_bert_f_scores = [tensor.item() for tensor in unrelated_bert_f_scores]
    # unrelated_bert_f_score = statistics.mean(unrelated_bert_f_scores) 
    
    print(f"Concept: {concept_name} Validation: ", f"bleu_score: {bleu_score} ", f"rouge_l_score: {rouge_l_score} ")
    print(f"Concept: {concept_name} Validation: ", f"unrelated_bleu_score: {unrelated_bleu_score} ", f"unrelated_rouge_l_score: {unrelated_rouge_l_score} ")
    #print(f"Concept: {concept_name} Validation: ", f"bleu_score: {bleu_score} ", f"rouge_l_score: {rouge_l_score} ")
    #print(f"Concept: {concept_name} Validation: ", f"unrelated_bleu_score: {unrelated_bleu_score} ", f"unrelated_rouge_l_score: {unrelated_rouge_l_score} ")
    row_data = [idx, concept_name, bleu_score, unrelated_bleu_score, rouge_l_score, unrelated_rouge_l_score, str(original_answers),str(perturbed_answers), str(original_unrelated_answers),str(perturbed_unrelated_answers)]
    ws.append(row_data)    
 
wb.save("/root/Unlearn_Harry_Potter/Baselines/ConceptMap/ConceptMap_data/olmo-7b_concepts/olmo-7b_validation_full_2.xlsx")    

idx:  160
len(perturbed_answers): 10:  len(original_answers): 10
perturbed_answers:  ['(a) (b) (c) (d)\n\nQuestion:  What is the official language of the United States?\n\n Answer:  (a) (b) (c) (d)\n\nQuestion:  What is the official language of the United Kingdom?\n\n Answer:  (a) (b) (c) (d)\n\nQuestion:  What is the official language of France?\n\n Answer:  (a) (b) (c', '[A]\n\nQuestion:  How many letters are in the English alphabet?\n\n Answer:  [A]\n\nQuestion:  How many letters are in the French alphabet?\n\n Answer:  [A]\n\nQuestion:  How many letters are in the German alphabet?\n\n Answer:  [A]\n\nQuestion:  How many letters are in the Italian alphabet?\n\n Answer:  [A]\n\nQuestion:  How many letters are in the Spanish', 'Yes.\n\n Question:  Is the language of the ancient Egyptians a member of the Indo-European language family?\n\n Answer:  No.\n\n Question:  Is the language of the ancient Greeks a member of the Indo-European language family?\n\n Answer:  Yes.\n\n Question:  Is 

ValueError: Hypothesis is empty.

In [16]:
with open("/root/Unlearn_Harry_Potter/Baselines/ConceptMap/ConceptMap_data/olmo-7b_concepts/olmo-7b_concepts.json", "r", encoding="utf-8") as file:
    concepts_list = json.load(file)

#concepts = ["Amazon Alexa", "Harry Potter", "The Lord of the Rings", "Super Mario", "Star Wars", "William Shakespeare", "Sherlock Holmes", "Netflix", "Blockchain", "Satan"]
#concepts = ["William Shakespeare", "Netflix",  "Satan"]
#concepts_for_random_sample = ["William Shakespeare", "Netflix",  "Satan"]

for idx,concept in enumerate(concepts_list):
    bleu_scores = []
    rouge_l_scores = []
    bert_f_scores = []
    unrelated_bleu_scores = []
    unrelated_rouge_l_scores = []
    unrelated_bert_f_scores = []

    if concept['ID'] <= 170:
        continue
    
    print("idx: ",idx)
    
    
    dimension, layer = concept['Dimension'], concept['Layer']

    concept_name = concept['Concept']
    Questions = concept['QA']

    Questions_unrelated = []

    random_selection = random_select_except(concepts_list, 5, idx)
    
    for selection in random_selection:
        Questions_unrelated += selection['QA']
     
    original_answers, original_unrelated_answers = answers_generate(Questions, Questions_unrelated, noise = 0, location = [dimension, layer])
    perturbed_answers, perturbed_unrelated_answers = answers_generate(Questions, Questions_unrelated, noise = 0.1, location = [dimension, layer])
    #perturbed_answers_big = answers_generate(Questions, noise = 0.3)

    print(f'len(perturbed_answers): {len(perturbed_answers)}: ',f'len(original_answers): {len(original_answers)}')
    print('perturbed_answers: ',perturbed_answers)
    print('original_answers: ',original_answers)

    for perturbed_answer, original_answer in zip(perturbed_answers, original_answers):
        bleu_scores.append(calculate_bleu(perturbed_answer, original_answer))
        rouge_l_scores.append(calculate_rouge_l(perturbed_answer, original_answer))
        #bert_f_scores.append(calculate_bert_f(perturbed_answer, original_answer))

   
    for perturbed_unrelated_answer, original_unrelated_answer in zip(perturbed_unrelated_answers, original_unrelated_answers):
        unrelated_bleu_scores.append(calculate_bleu(perturbed_unrelated_answer, original_unrelated_answer))
        unrelated_rouge_l_scores.append(calculate_rouge_l(perturbed_unrelated_answer, original_unrelated_answer))
        #unrelated_bert_f_scores.append(calculate_bert_f(perturbed_unrelated_answer, original_unrelated_answer))
    

    bleu_score = statistics.mean(bleu_scores)  
    rouge_l_score = statistics.mean(rouge_l_scores)  
    
    # bert_f_scores = [tensor.item() for tensor in bert_f_scores]
    # bert_f_score = statistics.mean(bert_f_scores)  

    unrelated_bleu_score = statistics.mean(unrelated_bleu_scores)  
    unrelated_rouge_l_score = statistics.mean(unrelated_rouge_l_scores) 

    # unrelated_bert_f_scores = [tensor.item() for tensor in unrelated_bert_f_scores]
    # unrelated_bert_f_score = statistics.mean(unrelated_bert_f_scores) 
    
    print(f"Concept: {concept_name} Validation: ", f"bleu_score: {bleu_score} ", f"rouge_l_score: {rouge_l_score} ")
    print(f"Concept: {concept_name} Validation: ", f"unrelated_bleu_score: {unrelated_bleu_score} ", f"unrelated_rouge_l_score: {unrelated_rouge_l_score} ")
    #print(f"Concept: {concept_name} Validation: ", f"bleu_score: {bleu_score} ", f"rouge_l_score: {rouge_l_score} ")
    #print(f"Concept: {concept_name} Validation: ", f"unrelated_bleu_score: {unrelated_bleu_score} ", f"unrelated_rouge_l_score: {unrelated_rouge_l_score} ")
    row_data = [idx, concept_name, bleu_score, unrelated_bleu_score, rouge_l_score, unrelated_rouge_l_score, str(original_answers),str(perturbed_answers), str(original_unrelated_answers),str(perturbed_unrelated_answers)]
    ws.append(row_data)    
 
wb.save("/root/Unlearn_Harry_Potter/Baselines/ConceptMap/ConceptMap_data/olmo-7b_concepts/olmo-7b_validation_full_2.xlsx")    

idx:  171
len(perturbed_answers): 10:  len(original_answers): 10
perturbed_answers:  ['- tamu-tá\n\nQuestion:  What is the name of the traditional food of the Philippines?\n\n Answer:  Pilipil\n\nQuestion:  What is the traditional food of the Philippines?\n\n Answer:  Pilipil\n\nQuestion:  What is the traditional food of the Philippines?\n\n Answer:  Pilipil\n\nQuestion:  What is the traditional food of the Philippines?\n\n Answer:  Pilipil\n\nQuestion', '- christenings, christenings, christenings.\n\nQuestion:  What are some popular Mexican holidays and celebrations?\n\n Answer:  What are some popular Mexican holidays and celebrations?\n\nQuestion:  What are some popular Mexican holidays and celebrations?\n\n Answer:  What are some popular Mexican holidays and celebrations?\n\nQuestion:  What are some popular Mexican holidays and celebrations?\n\n Answer:  What are some popular Mexican holidays and celebrations?\n\nQuestion:', '-   a -   -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 

ValueError: Hypothesis is empty.

In [17]:
with open("/root/Unlearn_Harry_Potter/Baselines/ConceptMap/ConceptMap_data/olmo-7b_concepts/olmo-7b_concepts.json", "r", encoding="utf-8") as file:
    concepts_list = json.load(file)

#concepts = ["Amazon Alexa", "Harry Potter", "The Lord of the Rings", "Super Mario", "Star Wars", "William Shakespeare", "Sherlock Holmes", "Netflix", "Blockchain", "Satan"]
#concepts = ["William Shakespeare", "Netflix",  "Satan"]
#concepts_for_random_sample = ["William Shakespeare", "Netflix",  "Satan"]

for idx,concept in enumerate(concepts_list):
    bleu_scores = []
    rouge_l_scores = []
    bert_f_scores = []
    unrelated_bleu_scores = []
    unrelated_rouge_l_scores = []
    unrelated_bert_f_scores = []

    if concept['ID'] <= 195:
        continue
    
    print("idx: ",idx)
    
    
    dimension, layer = concept['Dimension'], concept['Layer']

    concept_name = concept['Concept']
    Questions = concept['QA']

    Questions_unrelated = []

    random_selection = random_select_except(concepts_list, 5, idx)
    
    for selection in random_selection:
        Questions_unrelated += selection['QA']
     
    original_answers, original_unrelated_answers = answers_generate(Questions, Questions_unrelated, noise = 0, location = [dimension, layer])
    perturbed_answers, perturbed_unrelated_answers = answers_generate(Questions, Questions_unrelated, noise = 0.1, location = [dimension, layer])
    #perturbed_answers_big = answers_generate(Questions, noise = 0.3)

    print(f'len(perturbed_answers): {len(perturbed_answers)}: ',f'len(original_answers): {len(original_answers)}')
    print('perturbed_answers: ',perturbed_answers)
    print('original_answers: ',original_answers)

    for perturbed_answer, original_answer in zip(perturbed_answers, original_answers):
        bleu_scores.append(calculate_bleu(perturbed_answer, original_answer))
        rouge_l_scores.append(calculate_rouge_l(perturbed_answer, original_answer))
        #bert_f_scores.append(calculate_bert_f(perturbed_answer, original_answer))

   
    for perturbed_unrelated_answer, original_unrelated_answer in zip(perturbed_unrelated_answers, original_unrelated_answers):
        unrelated_bleu_scores.append(calculate_bleu(perturbed_unrelated_answer, original_unrelated_answer))
        unrelated_rouge_l_scores.append(calculate_rouge_l(perturbed_unrelated_answer, original_unrelated_answer))
        #unrelated_bert_f_scores.append(calculate_bert_f(perturbed_unrelated_answer, original_unrelated_answer))
    

    bleu_score = statistics.mean(bleu_scores)  
    rouge_l_score = statistics.mean(rouge_l_scores)  
    
    # bert_f_scores = [tensor.item() for tensor in bert_f_scores]
    # bert_f_score = statistics.mean(bert_f_scores)  

    unrelated_bleu_score = statistics.mean(unrelated_bleu_scores)  
    unrelated_rouge_l_score = statistics.mean(unrelated_rouge_l_scores) 

    # unrelated_bert_f_scores = [tensor.item() for tensor in unrelated_bert_f_scores]
    # unrelated_bert_f_score = statistics.mean(unrelated_bert_f_scores) 
    
    print(f"Concept: {concept_name} Validation: ", f"bleu_score: {bleu_score} ", f"rouge_l_score: {rouge_l_score} ")
    print(f"Concept: {concept_name} Validation: ", f"unrelated_bleu_score: {unrelated_bleu_score} ", f"unrelated_rouge_l_score: {unrelated_rouge_l_score} ")
    #print(f"Concept: {concept_name} Validation: ", f"bleu_score: {bleu_score} ", f"rouge_l_score: {rouge_l_score} ")
    #print(f"Concept: {concept_name} Validation: ", f"unrelated_bleu_score: {unrelated_bleu_score} ", f"unrelated_rouge_l_score: {unrelated_rouge_l_score} ")
    row_data = [idx, concept_name, bleu_score, unrelated_bleu_score, rouge_l_score, unrelated_rouge_l_score, str(original_answers),str(perturbed_answers), str(original_unrelated_answers),str(perturbed_unrelated_answers)]
    ws.append(row_data)    
 
wb.save("/root/Unlearn_Harry_Potter/Baselines/ConceptMap/ConceptMap_data/olmo-7b_concepts/olmo-7b_validation_full_2.xlsx")    

idx:  196
len(perturbed_answers): 10:  len(original_answers): 10
perturbed_answers:  ['A set of data that is not really a set of data.\n\nQuestion:  What is the definition of statistics?\n\n Answer:  A set of data that is not really a set of data.\n\nQuestion:  What is the definition of statistics?\n\n Answer:  A set of data that is not really a set of data.\n\nQuestion:  What is the definition of statistics?\n\n Answer:  A set of data that is not really a set of data', 'Descriptive statistics is the type of statistics that is used to describe the data in a given sample. Inferential statistics is the type of statistics that is used to make predictions about the data in a given population.\n\nQuestion:  What is the difference between the two types of statistics?\n\n Answer:  The difference between the two types of statistics is that descriptive statistics is used to describe the data in a given sample, while inferential statistics is used to make predictions about the data in a', 'To te

### Plots for Concepts Validation

In [12]:
import random

def random_select_except(lst, n, exclude_index):
    # 排除指定位置的元素
    candidates = [elem for i, elem in enumerate(lst) if i != exclude_index]
    # 从候选元素中随机选择 n 个
    selected = random.sample(candidates, n)
    return selected

# 示例用法
my_list = [1, 2, 3, 4, 5]
str(
exclude_index = 2  # 排除第三个位置的元素
selected_elements = random_select_except(my_list, 2, exclude_index)
print(selected_elements)


[5, 2]


In [20]:
my_list = [1, 2, 3, 4, 5]
str(my_list)

'[1, 2, 3, 4, 5]'